In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import scipy.special

In [ ]:
data = pd.read_csv('../input/lol-2021-competitive-game-from-012021-to-msi/2021_LoL_esports_match_data_from_OraclesElixir_20210504.csv', sep=';')

### Part one : Preliminary Dataset Analysis

Let's have a quick sight into the dataset before using it for prediction.

In [ ]:
data

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.columns

In [ ]:
data.describe()

**As we can see here, some columns are missing sata due to the collection process. This could be fixed by removing the rows where missing values occur or by using an updated version of the file.**

### Part Two : Win rates analysis


In [ ]:
clear_data = data[['league','t1_playerid','t1_result','t2_result','t2_playerid']]
clear_data

In [ ]:
clear_data.describe()

Here we can notice that for 2 rows there is missing values for t2_result. This is not big issue because t1_result implies t2_result but it shows that the data are not fully reliable still it is not purified. That is why the preliminary analysis is mandatory if you want to provide relevant and reliable results.

In [ ]:
data_bluewin = data[data.t1_result == 1]
data_bluewin

In [ ]:
averagewinblue = data.groupby('t1_playerid').mean()
bluewr = pd.DataFrame(averagewinblue['t1_result'])
sbluewr = bluewr.sort_values(by="t1_result")
sbluewr

This visualization shows us that some team does not have info for t1_result that means there is an error in their dataset rows. No interpretation can be done on those teams.

In [ ]:
sbluewrplt = sbluewr.plot(kind='bar',figsize=(20,10))
plt.ylabel("%")
axes = plt.gca()
axes.set_ylim(0, 1)
plt.show()

**The density of team on this graph is too high. So let's check the major regions only**

In [ ]:
LPLdata = data[['league','t1_playerid','t1_result','t2_result','t2_playerid']].loc[data.league=='LPL']
LECdata = data[['league','t1_playerid','t1_result','t2_result','t2_playerid']].loc[data.league=='LEC']
LCSdata = data[['league','t1_playerid','t1_result','t2_result','t2_playerid']].loc[data.league=='LCS']
LCKdata = data[['league','t1_playerid','t1_result','t2_result','t2_playerid']].loc[data.league=='LCK']
CBLOLdata = data[['league','t1_playerid','t1_result','t2_result','t2_playerid']].loc[data.league=='CBLOL']
TCLdata = data[['league','t1_playerid','t1_result','t2_result','t2_playerid']].loc[data.league=='TCL']
PCSdata = data[['league','t1_playerid','t1_result','t2_result','t2_playerid']].loc[data.league=='PCS']
LCLdata = data[['league','t1_playerid','t1_result','t2_result','t2_playerid']].loc[data.league=='LCL']
LJLdata = data[['league','t1_playerid','t1_result','t2_result','t2_playerid']].loc[data.league=='LJL']
LLAdata = data[['league','t1_playerid','t1_result','t2_result','t2_playerid']].loc[data.league=='LLA']
OPLdata = data[['league','t1_playerid','t1_result','t2_result','t2_playerid']].loc[data.league=='OPL']
PROdata = pd.concat([LPLdata, LECdata, LCSdata,LCKdata, TCLdata, CBLOLdata, LCLdata, PCSdata], axis=0)
PROdata

In [ ]:
PROaveragewinblue = PROdata.groupby('t1_playerid').mean()
PRObluewr = pd.DataFrame(PROaveragewinblue['t1_result'])
sPRObluewr = PRObluewr.sort_values(by="t1_result")
sPRObluewr

In [ ]:
sPRObluewrplt = sPRObluewr.plot(kind='bar',figsize=(20,10))
plt.ylabel("%")
axes = plt.gca()
axes.set_ylim(0, 1)
plt.show()

We can see here that PSG TAlon is the best team in term of winrate for blue side with 100% winrate. Aurora, Squadron and CTRL Play seem to be the worst.

**Let's do the same for red side now.**

In [ ]:
PROaveragewinred = PROdata.groupby('t2_playerid').mean()
PROredwr = pd.DataFrame(PROaveragewinred['t2_result'])
sPROredwr = PROredwr.sort_values(by="t2_result")
sPROredwr

In [ ]:
sPROredwrplt = sPROredwr.plot(kind='bar',figsize=(20,10), color ='red')
plt.ylabel("%")
axes = plt.gca()
axes.set_ylim(0, 1)
plt.show()

PSG Talon is still the best team (looks logical when you know they have done a 26-1 spring split).

In [ ]:
PROwr = pd.concat([sPRObluewr,sPROredwr], axis=1)
PROwr.plot(kind='bar',figsize=(20,10))
plt.ylabel("%")
axes = plt.gca()
axes.set_ylim(0, 1)
plt.show()

In [ ]:
PROwr

In [ ]:
PRObluegames = PROdata.groupby('t1_playerid').count()
PRObg = pd.DataFrame(PRObluegames['t1_result'])
PROredgames = PROdata.groupby('t2_playerid').count()
PROrg = pd.DataFrame(PROredgames['t2_result'])
PROsumgames = PRObluegames + PROredgames
PROsg = pd.DataFrame(PROsumgames['league'])
PROgames = pd.concat([PRObg,PROrg, PROsg], axis=1)
PROgames

In [ ]:
PROgwr = pd.DataFrame((PROwr.t1_result * PROgames.t1_result + PROwr.t2_result * PROgames.t2_result)/PROgames.league)
PROgwr = PROgwr.sort_values(by=0)
PROgwr

In [ ]:
PROgwr.plot(kind='bar',figsize=(20,10))
plt.ylabel("%")
axes = plt.gca()
axes.set_ylim(0, 1)
plt.show()

**Here we have the general winrate of teams for the early 2021 season.**

We can notice that the teams with the highest winrates are not always the ones qualified to the MSI. This could give info on what to analyze to have better predictions.
Finally, this preliminary and basic analysis is not sufficient to predict who will win MSI. I let you find way to optimize your predictions.